# Save and Restore
In this post we are going to talk about how to save the parameters into the disk and restore the saved parameters from the disk. The savable/restorable paramters of the network are __Variables__ (i.e. weights and biases).

## TLDR: 

To save and restore your variables, all you need to do is to call the `tf.train.Saver()` at the end of you graph.

```python
# create the graph
X = tf.placeholder(..)
Y = tf.placeholder(..)
w = tf.get_variable(..)
b = tf.get_variable(..)
...
loss = tf.losses.mean_squared_error(..)
optimizer = tf.train.AdamOptimizer(..).minimize(loss)
...

saver = tf.train.Saver()
```


__In the train mode__, in the session we will initialize the variables and run our network. At the end of training, we will save the variables using `saver.save()`:

```python
# TRAIN
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # train our model
    for step in range(steps):
        sess.run(optimizer)
        ...
    saved_path = saver.save(sess, './my-model', global_step=step)
```

This will create 3 files (`data`, `index`, `meta`) with a suffix of the step you saved your model.

__In the test mode__, in the session we will restore the variables using `saver.restore()` and validate or test our model.

```python
# TEST
with tf.Session() as sess:
    saver.restore(sess, './my-model')
    ...
```


## 1. Save and Restore Two Variables:
### 1.1 Save:
We will start with saving and restoring two variables in TensorFlow. We will create a graph with two variables. Let's create two variables `a = [3 3]` and `b = [5 5 5]`:

In [1]:
import tensorflow as tf
# create variables a and b
a = tf.get_variable("A", initializer=tf.constant(3, shape=[2]))
b = tf.get_variable("B", initializer=tf.constant(5, shape=[3]))

Notice the __lower__case letter as python name and __UPPER__case letter as TensorFlow name. It will be important when we want to import the graph in restoring the data. 

__Recall from the [Tensor Types Tutorial](https://github.com/easy-tensorflow/easy-tensorflow/blob/master/1_TensorFlow_Basics/Tutorials/2_Tensor_Types.ipynb):__ Variables need to be initialized before being used. To do so, we have to invoke a __variable initializer operation__ and run the operation on the session. This is the easiest way to initialize variables which initializes all variables at once.

In [2]:
# initialize all of the variables
init_op = tf.global_variables_initializer()

Now, on the session, we can initialize the variables and run the to see the values:

In [3]:
# run the session
with tf.Session() as sess:
    # initialize all of the variables in the session
    sess.run(init_op)
    # run the session to get the value of the variable
    a_out, b_out = sess.run([a, b])
    print('a = ', a_out)
    print('b = ', b_out)

a =  [3 3]
b =  [5 5 5]


__Important Note:__ All of the variables exist in the scope of the session. So, after the session is closed, we will loose the variable. 

In order to save the variable, we will call the saver function using `tf.train.Saver()` in our graph. This function will find all the variables in the graph. We can see the list of all variables in `_var_list`. Let's create a `saver` object and take a look at the `_var_list` in the object:

In [4]:
# create saver object
saver = tf.train.Saver()
for i, var in enumerate(saver._var_list):
    print('Var {}: {}'.format(i, var))

Var 0: <tf.Variable 'A:0' shape=(2,) dtype=int32_ref>
Var 1: <tf.Variable 'B:0' shape=(3,) dtype=int32_ref>


So, our graph consists of two variables that listed above.

__Important Note__: Notice the `:0` at the end of the variable name. For more about tensor naming check [here](https://stackoverflow.com/questions/36150834/how-does-tensorflow-name-tensors).

Now that the saver object is created in the graph, in the session, we can call the `saver.save()` function to save the variables in the disk. We have to pass the created session (`sess`) and the path to the file that we want to save the variables:

In [5]:
# run the session
with tf.Session() as sess:
    # initialize all of the variables in the session
    sess.run(init_op)
    
    # save the variable in the disk
    saved_path = saver.save(sess, './saved_variable')
    print('model saved in {}'.format(saved_path))

model saved in ./saved_variable


If you check your working directory, you will notice that 3  new files have been created with the name `saved_variable` in them.

In [6]:
import os
for file in os.listdir('.'):
    if 'saved_variable' in file:
        print(file)

saved_variable.data-00000-of-00001
saved_variable.meta
saved_variable.index


__.data:__ Contains variable values

__.meta:__ Contains graph structure

__.index:__ Identifies checkpoints

### 1.2. Restore:
Now that all the things that you need is saved in the disk, you can load your saved variables in the session using `saver.restore()`:

In [7]:
# run the session
with tf.Session() as sess:
    # restore the saved vairable
    saver.restore(sess, './saved_variable')
    # print the loaded variable
    a_out, b_out = sess.run([a, b])
    print('a = ', a_out)
    print('b = ', b_out)

INFO:tensorflow:Restoring parameters from ./saved_variable
a =  [3 3]
b =  [5 5 5]


Notice that this time we did not initialize the variables in our session. Instead, we restored them from the disk.

__Important Note:__ In order to restore the parameters, the graph should be defined. Since we defined the graph in top, we didn't have a problem restoring the parameters. But what happens if we have not loaded the graph?

In [8]:
# delete the current graph
tf.reset_default_graph()
try:
    with tf.Session() as sess:
        # restore the saved vairable
        saver.restore(sess, './saved_variable')
        # print the loaded variable
        a_out, b_out = sess.run([a, b])
        print('a = ', a_out)
        print('b = ', b_out)
except Exception as e:
    print(str(e))

INFO:tensorflow:Restoring parameters from ./saved_variable
The Session graph is empty.  Add operations to the graph before calling run().


We can define the graph in two ways.

#### 1.2.1. Define the graph from scratch and then run the session:
This way is simple if you have your graph. So, what you can  do is to create the graph and then restore your variables:

In [9]:
# delete the current graph
tf.reset_default_graph()

# create a new graph
# create variables a and b
a = tf.get_variable("A", initializer=tf.constant(3, shape=[2]))
b = tf.get_variable("B", initializer=tf.constant(5, shape=[3]))

# initialize all of the variables
init_op = tf.global_variables_initializer()

# create saver object
saver = tf.train.Saver()

# run the session
with tf.Session() as sess:
    # restore the saved vairable
    saver.restore(sess, './saved_variable')
    # print the loaded variable
    a_out, b_out = sess.run([a, b])
    print('a = ', a_out)
    print('b = ', b_out)

INFO:tensorflow:Restoring parameters from ./saved_variable
a =  [3 3]
b =  [5 5 5]


Keep in mind that the graph should be exactly like the one that you saved. ButwWhat if we do not know the exact graph and we are using someone else's pre-trained model?

#### 1.2.2. Restore the graph from `.meta` file.

When we save the variables, it creates a `.meta` file. This file contains the graph structure. Therefore, we can import the meta graph using `tf.train.import_meta_graph()` and restore the values of the graph. Let's import the graph and see all tensors in the graph:

In [10]:
# delete the current graph
tf.reset_default_graph()

# import the graph from the file
imported_graph = tf.train.import_meta_graph('saved_variable.meta')

# list all the tensors in the graph
for tensor in tf.get_default_graph().get_operations():
    print (tensor.name)

Const
A
A/Assign
A/read
Const_1
B
B/Assign
B/read
init
save/Const
save/SaveV2/tensor_names
save/SaveV2/shape_and_slices
save/SaveV2
save/control_dependency
save/RestoreV2/tensor_names
save/RestoreV2/shape_and_slices
save/RestoreV2
save/Assign
save/RestoreV2_1/tensor_names
save/RestoreV2_1/shape_and_slices
save/RestoreV2_1
save/Assign_1
save/restore_all


If you recall from section 1.1, we defined the python names with __lower__case letters and in TensorFlow names with __UPPER__case letters. You can see that what we have here are the __UPPER__case letter variables. It means that `tf.train.Saver()` saves the variables with the TensorFlow name. Now that we have the imported graph, and we know that we are interested in `A` and `B` tensors, we can restore the parameters:

In [11]:
# run the session
with tf.Session() as sess:
    # restore the saved vairable
    imported_graph.restore(sess, './saved_variable')
    # print the loaded variable
    a_out, b_out = sess.run(['A:0','B:0'])
    print('a = ', a_out)
    print('b = ', b_out)

INFO:tensorflow:Restoring parameters from ./saved_variable
a =  [3 3]
b =  [5 5 5]


__Important Note:__ Notice that in `sess.run()` we provided  the TensorFlow name of the tensors `'A:0'` and `'B:0'` instead of `a` and `b`. 

## 2. Save and Restore Variables of a Sample Linear Model:
Now that we have learnt how to save and restore parameters, we can write a simple model and try to save and restore the __weights__ and __biases__ in this network.

We will build a simple linear model. If you do not know about the linear model, check our [Linear Classifier Tutorial](https://github.com/easy-tensorflow/easy-tensorflow/tree/master/2_Linear_Classifier).

In [12]:
# delete the current graph
tf.reset_default_graph()

# Data Dimensions
img_h = img_w = 28              # MNIST images are 28x28
img_size_flat = img_h * img_w   # 28x28=784, the total number of pixels
n_classes = 10                  # Number of classes, one class per digit

# Load MNIST data
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("MNIST/", one_hot=True)

# Hyper-parameters
learning_rate = 0.001   # The optimization initial learning rate
batch_size = 100        # Training batch size
num_steps = 1000         # Total number of training steps

# Placeholders for inputs (x), outputs(y)
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='X')
y = tf.placeholder(tf.float32, shape=[None, n_classes], name='Y')

W = tf.get_variable('W',
                    dtype=tf.float32,
                    shape=[img_size_flat, n_classes],
                    initializer=tf.truncated_normal_initializer(stddev=0.01))
b = tf.get_variable('b',
                    dtype=tf.float32,
                    initializer=tf.constant(0., shape=[n_classes], dtype=tf.float32))

# Calculate the output logits as: output_logits = W*x + b
output_logits = tf.matmul(x, W) + b
# Convert logits to probabilities
y_pred = tf.nn.softmax(output_logits)

# Define the loss function, optimizer, and accuracy
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output_logits), name='loss')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op').minimize(loss)
correct_prediction = tf.equal(tf.argmax(output_logits, 1), tf.argmax(y, 1), name='correct_pred')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


At the end of graph, will call the `tf.train.Saver()` to save all the variables.

In [13]:
# create saver object
saver = tf.train.Saver()

Now we can run the model and save the variables.

In [14]:
# run the session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_steps):
        # Get a batch of training examples and their corresponding labels.
        x_batch, y_true_batch = data.train.next_batch(batch_size)

        # Put the batch into a dict to be fed into the placeholders
        feed_dict_train = {x: x_batch, y: y_true_batch}
        sess.run(optimizer, feed_dict=feed_dict_train)

    feed_dict_valid = {x: data.validation.images, y: data.validation.labels}
    loss_test, acc_test = sess.run([loss, accuracy], feed_dict=feed_dict_valid)
    print('---------------------------------------------------------')
    print("Validation loss: {0:.2f}, Validation accuracy: {1:.01%}".format(loss_test, acc_test))
    print('---------------------------------------------------------')

    # save the variable in the disk
    saved_path = saver.save(sess, './linear_model')
    print('model saved in {}'.format(saved_path))

---------------------------------------------------------
Validation loss: 0.31, Validation accuracy: 91.7%
---------------------------------------------------------
model saved in ./linear_model


We can check that the model is saved in `./linear_model`.

In [15]:
for file in os.listdir('.'):
    if 'linear_model' in file:
        print(file)

linear_model.data-00000-of-00001
linear_model.meta
linear_model.index


Let's restore the model and pull out the trained variables. at this time, the garph still exists in the memory. So, we can restore it and evaluate the network on the test set:

In [16]:
# Test the network after training

# run the session
with tf.Session() as sess:
    # restore the saved vairable
    imported_graph.restore(sess, './linear_model')
    
    # Accuracy
    feed_dict_test = {x: data.test.images, y: data.test.labels}
    loss_test, acc_test = sess.run([loss, accuracy], feed_dict=feed_dict_test)
    print('---------------------------------------------------------')
    print("Test loss: {0:.2f}, test accuracy: {1:.01%}".format(loss_test, acc_test))
    print('---------------------------------------------------------')
    print()

    
    # print the loaded variable
    weight, bias = sess.run(['W:0','b:0'])
    print('W = ', weight)
    print('b = ', bias)
    


INFO:tensorflow:Restoring parameters from ./linear_model
---------------------------------------------------------
Test loss: 0.32, test accuracy: 91.4%
---------------------------------------------------------

W =  [[ 0.01166079  0.00214     0.01031407 ...  0.00138819 -0.00628771
   0.00202227]
 [ 0.00583515 -0.0049552   0.00155843 ... -0.00023047 -0.00729254
  -0.00405982]
 [ 0.01768233 -0.00875011 -0.01097609 ... -0.00147737 -0.00018261
  -0.00989295]
 ...
 [ 0.00241719  0.00672442 -0.00274725 ...  0.01378055 -0.00826664
  -0.00141329]
 [-0.01537914 -0.00139711  0.0020227  ... -0.00358674  0.00436257
  -0.00737171]
 [ 0.00428195  0.00458214 -0.01005485 ...  0.00464132  0.00323565
  -0.00285462]]
b =  [-0.14392368  0.2330784  -0.04319457 -0.10059926  0.06679667  0.21008638
 -0.0314009   0.14653793 -0.30130398 -0.05041829]


Recall from __Section 1.2__, if we do not have the graph, we can restore the values of the graph using `tf.train.import_meta_graph()`:

In [17]:
# delete the current graph
tf.reset_default_graph()

# import the graph from the file
imported_graph = tf.train.import_meta_graph('linear_model.meta')

# list all the tensors in the graph
for tensor in tf.get_default_graph().get_operations():
    print (tensor.name)

X
Y
W/Initializer/truncated_normal/shape
W/Initializer/truncated_normal/mean
W/Initializer/truncated_normal/stddev
W/Initializer/truncated_normal/TruncatedNormal
W/Initializer/truncated_normal/mul
W/Initializer/truncated_normal
W
W/Assign
W/read
Const
b
b/Assign
b/read
MatMul
add
Softmax
softmax_cross_entropy_with_logits/Rank
softmax_cross_entropy_with_logits/Shape
softmax_cross_entropy_with_logits/Rank_1
softmax_cross_entropy_with_logits/Shape_1
softmax_cross_entropy_with_logits/Sub/y
softmax_cross_entropy_with_logits/Sub
softmax_cross_entropy_with_logits/Slice/begin
softmax_cross_entropy_with_logits/Slice/size
softmax_cross_entropy_with_logits/Slice
softmax_cross_entropy_with_logits/concat/values_0
softmax_cross_entropy_with_logits/concat/axis
softmax_cross_entropy_with_logits/concat
softmax_cross_entropy_with_logits/Reshape
softmax_cross_entropy_with_logits/Rank_2
softmax_cross_entropy_with_logits/Shape_2
softmax_cross_entropy_with_logits/Sub_1/y
softmax_cross_entropy_with_logits/Su

Let's say that I am interested in `loss` and `accuracy` of my model. We can easily get the values of corresponding tensors, providing the correct placeholders:

In [18]:
# run the session
with tf.Session() as sess:
    # restore the saved vairable
    imported_graph.restore(sess, './linear_model')
        
    # Accuracy
    feed_dict_test = {'X:0': data.test.images, 'Y:0': data.test.labels}
    loss_test, acc_test = sess.run(['loss:0', 'accuracy:0'], feed_dict=feed_dict_test)
    print('---------------------------------------------------------')
    print("Test loss: {0:.2f}, test accuracy: {1:.01%}".format(loss_test, acc_test))
    print('---------------------------------------------------------')
    print()

INFO:tensorflow:Restoring parameters from ./linear_model
---------------------------------------------------------
Test loss: 0.32, test accuracy: 91.4%
---------------------------------------------------------



Thanks for reading! If you have any question or doubt, feel free to leave a comment in our [website](http://easy-tensorflow.com/).